In [1]:
import os
os.chdir('../')


In [2]:

import sys 
sys.path.append('../')
import time
import pandas as pd 
import numpy as np
import torch
import openai
from tqdm import tqdm
from dotenv import load_dotenv
from trainer.transformer_utilts import *
from helper.dataloader import * 
import pickle
from peft import get_peft_model
from model.MF import sentenceT5Classification
import argparse
from tqdm import tqdm
from peft import LoraConfig, TaskType
import logging
args = parse_args(notebook=True)



/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


MODEL NAME = /home/mila/e/emiliano.penaloza/scratch/saved_model/ml-1m/Transformer/best_model_lr_0.0001_embedding_module_microsoft_phi-2_epochs_3_l2_lambda_0.0001_lora_alpha_16_lora_r_16_bce_softmax__bias_False.pth
args.model_log_name='Transformer_ml-1m_embedding_module_microsoft_phi-2_2024-06-11_12-31-12_2024.csv'


In [4]:
id_title_map = map_id_to_title(args.data_name)
id_genre = map_id_to_genre(args.data_name)
genre_title_map = {}
for movie_id in id_genre:
    genre = id_genre[movie_id]
    title = id_title_map[movie_id]
    genre_title_map[title] = genre
    


In [3]:
from model.MF import *
tokenizer = get_tokenizer(args)
args.bs = 250
args.data_name = 'goodbooks'
prompts,rec_dataloader,augmented_dataloader,num_movies,val_dataloader,test_dataloader,val_data_tr,test_data_tr= load_data(args,tokenizer,0,1)


/home/mila/e/emiliano.penaloza/llm4rec/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:246: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on google-t5/t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing c

Max Prompt Length 246
Number of Users is num_users=9980
Number of Movies is num_movies=8093


In [4]:
movie_list = [f" movie: {movie} id : {id}\n " for id ,movie in id_title_map.items()]



NameError: name 'id_title_map' is not defined

In [ ]:
load_dotenv()
key = os.getenv("OPEN-AI-SECRET")
openai.api_key = key = os.getenv("OPEN-AI-SECRET")
k= 50

In [19]:
import pandas as pd
import logging
# Set up logging
logging.basicConfig(filename='log.txt', level=logging.INFO)

message = []
k_values = [10, 20, 50]
counter = 0 
metrics = defaultdict(list)
try:
    starting_df = pd.read_csv(f'/home/mila/e/emiliano.penaloza/LLM4REC/results/peft_openai.csv_{args.data_name}')
    completed_users = len(starting_df)
except Exception: 
completed_users = 0 
item_type = 'books'
print(f"{completed_users=}")
for b in test_dataloader:
    user_ids = b['idx']

    preds = torch.zeros_like(b['labels'])

    k = 100
    gpt_outputs = {}
    movies_dict = {}
    for i, user_id in (pbar:=tqdm( enumerate(user_ids))):
        if i < completed_users:
            continue
        not_done = True
        while not_done:
            user_c = 0
            user_summary = prompts[user_id.item()]
            labels = b['labels']
            ind = torch.nonzero(b['labels_tr'][i]).flatten()
            seen_movies = '\n'.join([id_title_map[x.item()] for x in ind][:25])




            prompt = f'''
            
            User summary: 

            {user_summary}

            Here are the available {item_type}: 

            {"".join(movie_list[:-3000])}

            Important, do not recommend the following {item_type} as they have already been seen by the user:
            
            {seen_movies}
            
            Please only output the top {k} {item_type}. Simply print their id do not use the title out of a total of {len(movie_list)} {item_type} in the format:
            id1, id2, ... idn
            
            '''

   

            system_prompt = f'''
            You are a movie recommendation system, please recommendation system, please recommend a movie based on the following user preferences
            

            '''
            msg = [
                        {
                            "role": "system",
                            "content": system_prompt

                        },
                        {
                            "role": "user",
                            "content": prompt
                        },                
            ]
            outputs = openai.ChatCompletion.create(
                        model='gpt-4-1106-preview',
                        messages=msg,
                        # max_tokens=300,
                        # temperature=1,
                        seed=2024,
                    )['choices'][0]['message']['content']

            try:
                movies =[int(x) for x in  outputs.split(',')]
                movies_dict[user_id.item()] = movies
                gpt_outputs[user_id.item()] = outputs
                max_val = len(movies)
                for j,mov in enumerate(movies):
                    
                    preds[i,mov] = (max_val - j) + 1 
                #map back to movie title 
                movie_titles = [id_title_map[movie] for movie in movies]
                
                
                message.append(msg)
                not_done = False
            except Exception:
                counter += 1 
                user_c += 1 
                not_done = False
                if user_c > 5:
                    break
                print(f"{counter=} {user_c=}")
            
        if user_c > 5:
            for k in k_values:
                ndcg = NDCG_binary_at_k_batch(preds, labels, k=k).tolist()
                ndcg = [0 if iteration > i else x for iteration,x in enumerate(ndcg)]
                print(f"{ndcg=}")
                metrics[f'ndcg@{k}'].append(0)
                # metrics[f'mrr@{k}'].append(MRR_at_k(preds, labels, k=k,mean = False).tolist())
                recall = Recall_at_k_batch(preds, labels, k=k,mean = False).tolist()
                recall = [0 if iteration > i else x for iteration,x in enumerate(recall)]
                metrics[f'recall@{k}'].append(0)
                print('appending_zero')
        else:
            labels = labels.cpu().numpy()
            preds = preds
            for k in k_values:
                    ndcg = NDCG_binary_at_k_batch(preds, labels, k=k).tolist()
                    ndcg = [0 if iteration > i else x for iteration,x in enumerate(ndcg)]
                    print(f"{ndcg=}")
                    metrics[f'ndcg@{k}'].append(ndcg)
                    # metrics[f'mrr@{k}'].append(MRR_at_k(preds, labels, k=k,mean = False).tolist())
                    recall = Recall_at_k_batch(preds, labels, k=k,mean = False).tolist()
                    recall = [0 if iteration > i else x for iteration,x in enumerate(recall)]
                    metrics[f'recall@{k}'].append(recall)
            pbar.set_description(f"counter: {np.mean(metrics['ndcg@50'])} {np.mean(metrics['recall@50'])} {np.mean(metrics['recall@20'])} {counter}")
                

        metrics_df = pd.DataFrame(metrics)
        #log the outputs and movies to a dataframe as well 
        outputs_df = pd.DataFrame(gpt_outputs.items(),columns=['user_id','output'])
        movies_df = pd.DataFrame(movies_dict.items(),columns=['user_id','movies'])
        #merge into one dataframe 
        outputs_df = outputs_df.merge(movies_df,on='user_id')
        #save 
        outputs_df.to_csv(f'results/peft_openai_outputs_{args.data_name}.csv', index=False, mode='a',header = False)

        # save the metrics dataframe 
        metrics_df.to_csv(f'results/peft_openai.csv_{args.data_name}', index=False, mode='a',header = False)
        
                

completed_users=0


counter: 0.0 0.0 0.0 0: : 1it [00:56, 56.87s/it]

ndcg=[0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg=[0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

counter: 0.0 0.0 0.0 0: : 2it [01:20, 37.11s/it]

ndcg=[0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ndcg=[0.0, 0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

...

ndcg=[0.0, 0.0, 0.0, 0.0, 0.08514311764162101, 0.0, 0.0, 0.0, 0.0, 0.06625422345438904, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07336392209936006, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13886244387355456, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09478836436955079, 0.0, 0.0, 0.0, 0.09478836436955079, 0.0, 0.0, 0.0, 0.06943122193677728, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.20483424751859092, 0.24890832702259466, 0.0, 0.0, 0.2200917662980802, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29345568839744024, 0.0, 0.0, 0.13886244387355456, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06625422345438904, 0.0, 0.0, 0.0, 0.2200917662980802, 0.0, 0.0, 0.06943122193677728, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06943122193677728, 0.0, 0.06625422345438904, 0.0, 0.13886244387355456, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2200917662980802, 0.0

counter: 0.005555496603559771 0.005135153639610032 0.005336699507389162 36: : 202it [2:26:43, 50.13s/it]

ndcg=[0.0, 0.0, 0.0, 0.0, 0.08514311764162101, 0.0, 0.0, 0.0, 0.0, 0.06625422345438904, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07336392209936006, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.13886244387355456, 0.0, 0.0, 0.0, 0.0, 0.0, 0.09478836436955079, 0.0, 0.0, 0.0, 0.09478836436955079, 0.0, 0.0, 0.0, 0.06943122193677728, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.20483424751859092, 0.24890832702259466, 0.0, 0.0, 0.2200917662980802, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.29345568839744024, 0.0, 0.0, 0.13886244387355456, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06625422345438904, 0.0, 0.0, 0.0, 0.2200917662980802, 0.0, 0.0, 0.06943122193677728, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.06943122193677728, 0.0, 0.06625422345438904, 0.0, 0.13886244387355456, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2200917662980802, 0.0

counter: 0.005555496603559771 0.005135153639610032 0.005336699507389162 36: : 203it [2:26:48, 43.39s/it]


KeyboardInterrupt: 

In [11]:
df = pd.read_csv('/home/mila/e/emiliano.penaloza/LLM4REC/results/peft_openai_goodbooks.csv')


In [13]:
df.to_csv('/home/mila/e/emiliano.penaloza/LLM4REC/results/peft_openai_netflix.csv',index=False)

In [12]:
df

,"[0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0].1","[0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0].2","[0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0].3","[0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0].4","[0.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [24]:
import ast

recall20 = np.array(ast.literal_eval(df.iloc[-1,5]))

inds = [isinstance(x,float) for x in  ast.literal_eval(df.iloc[-1,1])]
print(f"{inds=}")
np.round(np.mean(recall20[inds]),3)



inds=[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True

0.011

In [32]:
metrics_df.iloc[-1,:].apply(lambda x: np.mean(x))

NameError: name 'metrics_df' is not defined

In [ ]:
preds = torch.zeros_like(b['labels'])

for j,mov in enumerate(movies):
                    
                    preds[i,mov] = (max_val - j) + 1 

In [ ]:
p = NDCG_binary_at_k_batch(preds, labels, k=k).tolist()
print(f"{preds.shape=}")
print(f"{labels.shape=}")

preds.shape=torch.Size([250, 3522])
labels.shape=(250, 3522)


In [ ]:
metrics_df = pd.DataFrame(metrics)


In [ ]:
metrics

defaultdict(list, {})

In [ ]:
for k in k_values:
            metrics[f'ndcg@{k}'].append(NDCG_binary_at_k_batch(preds.cpu().numpy(), labels.cpu().numpy(), k=k))
            metrics[f'mrr@{k}'].append(MRR_at_k(preds, labels, k=k,mean = False))
            # metrics[f'recall@{k}'].append(Recall_at_k_batch(preds, labels, k=k,mean = False))

In [ ]:
metrics

defaultdict(list,
            {'ndcg@10': [[0.0, 0.0, 0.0, 0.0], array([0., 0., 0., 0.])],
             'mrr@10': [torch.return_types.max(
              values=tensor([0., 0., 0., 0.], dtype=torch.float64),
              indices=tensor([0, 0, 0, 0]))],
             'ndcg@20': [array([0.        , 0.06818766, 0.        , 0.        ])],
             'mrr@20': [torch.return_types.max(
              values=tensor([0.0000, 0.0625, 0.0000, 0.0000], dtype=torch.float64),
              indices=tensor([ 0, 15,  0,  0]))],
             'ndcg@50': [array([0.        , 0.06721309, 0.08434198, 0.03693451])],
             'mrr@50': [torch.return_types.max(
              values=tensor([0.0000, 0.1000, 0.5000, 0.0357], dtype=torch.float64),
              indices=tensor([ 0,  9,  1, 27]))]})

In [ ]:
import torch

def recall_at_k(predictions, ground_truth, k):
    # Ensure k is smaller than the number of items
    k = min(k, ground_truth.size(1))

    # Get the top-k predicted items for each user
    _, topk_preds = torch.topk(predictions, k, dim=1)

    # Create a new zero tensor of the same shape as ground_truth
    topk_pred_mask = torch.zeros_like(predictions).scatter_(1, topk_preds, 1)

    # Compute the intersection of top k predictions and the ground truth
    correct_pred = topk_pred_mask * ground_truth

    # Summing the correct predictions and actual interactions for each user
    num_correct_pred = correct_pred.sum(dim=1)
    num_actual_interactions = ground_truth.sum(dim=1).clamp(max=k)

    # Recall@k for each user
    recall_at_k_per_user = num_correct_pred / num_actual_interactions

    # Handle division by zero for users with no actual interactions
    recall_at_k_per_user[num_actual_interactions == 0] = 0

    return recall_at_k_per_user.mean()  # Return the mean recall@k across all users

# Example usage:
# predictions = torch.tensor([[0.1, 0.2, 0.7], [0.4, 0.


In [ ]:
from sklearn.metrics import ndcg_score
from sklearn.metrics import ndcg_score
preds = torch.zeros_like(labels)
preds[:,movies] = 1
k = 10  # specify the value of k

ndcg = ndcg_score(labels, preds, k=50)

print(f"NDCG@{k}: {ndcg}")


NDCG@10: 0.0


[1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [ ]:
genres = openai.ChatCompletion.create(
                model='gpt-4-1106-preview',
                messages=msg,
                # max_tokens=300,
                # temperature=1,
                seed=2024,
            )['choices'][0]['message']['content']


In [ ]:
movies =[int(x) for x in  genres.split(',')]
#movie_idmap the movies 

#map back to movie title 
movie_titles = [id_title_map[movie] for movie in movies]


In [ ]:
movie_titles

50